# Dotrenowywanie modeli typu BERT

In [ ]:
!pip install -q transformers[torch] datasets
!pip install -q wandb
!pip install accelerate -U

In [ ]:
!pip install -q evaluate

In [ ]:
!wandb login
import wandb
import numpy as np
import evaluate

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer)


## Dataset

Ściągamy przygotowany wcześniej dataset [Yelp Reviews](https://huggingface.co/datasets/yelp_review_full):

Składa się z tekstu i z labelki:

In [ ]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

In [ ]:
print(dataset)

Żeby skrócić czas treningu tworzymy mniejszy dataset:

In [ ]:
N_TRAIN = 1000
N_VALID = 200
N_TEST = 200

small_train_set = dataset['train'].select([i for i in range(N_TRAIN)])
small_valid_set = dataset['train'].select([5000+i for i in range(N_VALID)])
small_test_set = dataset['test'].select([i for i in range(N_VALID)])

small_dataset = DatasetDict({
    'train': small_train_set,
    'test': small_test_set,
    'valid': small_valid_set})

In [ ]:
print(small_dataset)

Mamy tekst i labelki, ale teraz musimy jeszcze przygotować dane do przetwarzania przez transformery:

In [ ]:
max_length = 512
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=max_length)


tokenized_datasets = small_dataset.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:
sequence = "Where are we now? How many tokens are in seqence?"

bert_tokenized_sequence = tokenizer.tokenize(sequence)


print("BERT:", bert_tokenized_sequence)

## Trening

## Trening z wbudowaną klasą Trainer

Teraz pobieramy pretrenowany model z huggingface i określamy ile jest docelowych klas.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

### Ewaluacja

W trakcie treningu automatycznie liczymy tylko funkcje straty; jeśli chcemy mieć dodatkowe metryki musimy je dodać sami, na początek zaimplementujmy tylko dokładność

In [ ]:
metric = evaluate.load("accuracy")

I tworzymy metodę do liczenia dokładności:

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### Hiperparametery

Teraz definiujemy parametry treningu - w zależności jak trafnie je dobierzemy, tym lepsze uzyskamy wyniki

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=3,
                                  learning_rate=0.00002,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=8,
                                  weight_decay=0.001,
                                  disable_tqdm=False,
                                  overwrite_output_dir=True,
                                  metric_for_best_model='eval_loss',
                                  load_best_model_at_end=True,
                                  save_strategy='epoch',
                                  logging_strategy='epoch',
                                  log_level='error',
                                  warmup_ratio=0.05,
                                  report_to='wandb')

### Trainer

Huggingface używa osobnego obiektu, służącemu do treningu

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    compute_metrics=compute_metrics,
)

Dotrenowanie modelu uruchamiamy przy użyciu [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
wandb.init(project='distilber_reviews')
trainer.train()
preds_output = trainer.predict(tokenized_datasets['test'])
print(f"Test accuracy was {preds_output.metrics['test_accuracy']*100}%")
wandb_di = {'test_acc': preds_output.metrics['test_accuracy']}
wandb.log(wandb_di)
wandb.finish()


## Tokenizacja różnych modeli:

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
sequence = "Where are we now? Can we find rules used for tokenization, especially longer words such as honorificabilitudinitatibus?"

bert_tokenized_sequence = bert_tokenizer.tokenize(sequence)
roberta_tokenized_sequence = roberta_tokenizer.tokenize(sequence)

print("BERT:", bert_tokenized_sequence)
print("RoBERTa:", roberta_tokenized_sequence)

# Pipeline i NER

Trenowanie NERa to tak naprawdę clasyfikacja każdego tokenu:

![](https://ar5iv.labs.arxiv.org/html/1912.01389/assets/bert_arch.png)

Przykład, jak trenuje się takie datasety znajduje się [tutaj](https://huggingface.co/learn/nlp-course/chapter7/2)


Warto pamiętać, że można wykorzystać już wcześniej wytrenowane obiekty, jeśli użyjmey pipeline'u, sprawa staję się banalnie prosta.


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import pandas as pd

model_checkpoint = "pietruszkowiec/herbert-base-ner"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "Nazywam się Grzegorz Brzęszczyszczykiewicz, pochodzę "\
    "z Chrząszczyżewoszczyc, pracuję w Łękołodzkim Urzędzie Powiatowym"

ner_results = nlp(example)
results_df = pd.DataFrame(ner_results)
results_df.head(20)


## QA

Przy pomocy architektury BERT QA jest dość prymitywny, ale zobaczmy jak działa:

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/deberta-v3-large-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)
print(res)

# Zadanie

Znaleźć wytrenowany model do:
- klasyfikacji wydźwięku twittera w huggingface, wgrać go i sprawdzić jak działa na wybranych tekstach, można porównać kilka modeli
- do klasyfikacji toskycznych komentarzy w sieci i sprawdzić jak działa.